In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from catboost import CatBoostRegressor
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/raw/train.csv', parse_dates=['Date'])
sample_submission = pd.read_csv('data/raw/sample_submission.csv')

In [4]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
train['Month'] = train['Date'].dt.to_period('M').astype(str)

In [5]:
monthly_agg = (
    train
    .groupby(['Company_ID', 'Product_ID', 'Month'], as_index=False)
    .agg({'Target': 'sum'})
    .rename(columns={'Target': 'sum_sales'})
    .sort_values(by=['Company_ID', 'Product_ID', 'Month'])
)

In [6]:
d = {'Company_ID': [], 'Product_ID': [], 'Month': []}
ar = monthly_agg['Company_ID'].sort_values().unique()
br = monthly_agg['Product_ID'].sort_values().unique()
cr = monthly_agg['Month'].sort_values().unique()
for company_id in ar:
    for product_id in br:
        for month in cr:
            d['Company_ID'].append(company_id)
            d['Product_ID'].append(product_id)
            d['Month'].append(month)
d = pd.DataFrame(d)
monthly_agg = d.merge(monthly_agg, 'left', on=['Company_ID', 'Product_ID', 'Month']).fillna(0)

In [7]:
monthly_agg_saved = monthly_agg.copy()
monthly_agg

,Company_ID,Product_ID,Month,sum_sales
0,0,0,2019-01,0.0
1,0,0,2019-02,0.0
2,0,0,2019-03,0.0
3,0,0,2019-04,0.0
4,0,0,2019-05,0.0
...,...,...,...,...
3459115,3,14668,2023-08,0.0
3459116,3,14668,2023-09,0.0
3459117,3,14668,2023-10,0.0
3459118,3,14668,2023-11,0.0


In [8]:
monthly_agg['Month_date'] = pd.to_datetime(monthly_agg['Month'], format='%Y-%m')

n_lags = 15

subset = []
gb = monthly_agg.groupby(['Company_ID','Product_ID'])
monthly_agg['F'] = gb['sum_sales'].transform('std')
subset.append('F')
for i in range(n_lags + 1):
    monthly_agg[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
    subset.append(f'prev_sum_sales_{i}')
    if i > 0:
        if i + 1 <= 9:
            feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
            monthly_agg[f'rolling_sum_sales_{i + 1}'] = monthly_agg[feats_to_mean].mean(skipna=False, axis=1)
            subset.append(f'rolling_sum_sales_{i + 1}')
            monthly_agg[f'delta_regular_rolling_sum_sales_{i + 1}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'rolling_sum_sales_{i + 1}']
            subset.append(f'delta_regular_rolling_sum_sales_{i + 1}')
            if i > 1:
                monthly_agg[f'delta_rolling_sum_sales_{i}'] = monthly_agg[f'rolling_sum_sales_{i}'] - monthly_agg[f'rolling_sum_sales_{i + 1}']
                subset.append(f'delta_rolling_sum_sales_{i}')
        monthly_agg[f'delta_sum_sales_{i}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'prev_sum_sales_{i}']
        subset.append(f'delta_sum_sales_{i}')

monthly_agg['target'] = gb['sum_sales'].shift(-1)

In [9]:
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances

p = monthly_agg[monthly_agg['target'] != 0][subset + ['Product_ID']].dropna()

n_components_pca = 16
pca_cols = [f'PCA_{i}' for i in range(n_components_pca)]
pca = PCA(n_components=n_components_pca)
pca.fit(p[subset])

pca_features = pd.DataFrame(pca.transform(p[subset]), 
                            index=p.index, 
                            columns=pca_cols)
pca_features['Product_ID'] = p['Product_ID']

by = pca_features.groupby('Product_ID').mean()
distance_matrix = pd.DataFrame(np.log(euclidean_distances(by.values) + 1), index=by.index, columns=by.index)

clustering = DBSCAN(eps=0.5, min_samples=10, metric="precomputed")
clusters = pd.DataFrame({'Product_ID': distance_matrix.index, 'cluster': clustering.fit_predict(distance_matrix)})

In [10]:
distance_matrix

Product_ID,0,1,2,3,5,6,7,8,9,10,...,14659,14660,14661,14662,14663,14664,14665,14666,14667,14668
Product_ID,,,,,,,,,,,,,,,,,,,,,
0,0.000000,4.250418,3.143190,2.212673,1.786320,3.177325,0.934340,0.943663,1.311663,1.023655,...,4.928953,1.529135,0.892289,0.834719,4.319033,1.541313,5.646264,0.959294,1.150516,1.897503
1,4.250418,0.000000,3.874205,4.128365,4.182574,3.854815,4.267296,4.259115,4.214081,4.233854,...,4.241227,4.199902,4.264498,4.260038,2.290553,4.215072,5.410422,4.267022,4.261638,4.172026
2,3.143190,3.874205,0.000000,2.722461,2.937165,1.535092,3.196234,3.175586,3.035513,3.099157,...,4.755051,2.987874,3.188798,3.174378,3.973356,3.051584,5.578784,3.194454,3.186389,2.902744
3,2.212673,4.128365,2.722461,0.000000,1.592629,2.771144,2.336108,2.286041,1.910084,2.089731,...,4.869125,1.745450,2.316475,2.288378,4.205607,1.970407,5.620283,2.333350,2.310615,1.515546
5,1.786320,4.182574,2.937165,1.592629,0.000000,2.962264,1.940119,1.841572,1.285608,1.535859,...,4.895669,0.958250,1.903321,1.869838,4.257061,1.295810,5.628921,1.947537,1.875091,0.888351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14664,1.541313,4.215072,3.051584,1.970407,1.295810,3.071631,1.692573,1.496858,1.156234,1.180337,...,4.912111,1.211520,1.627491,1.638827,4.286546,0.000000,5.634624,1.700167,1.490007,1.527460
14665,5.646264,5.410422,5.578784,5.620283,5.628921,5.570185,5.649099,5.646172,5.637272,5.640924,...,5.181827,5.634122,5.648139,5.647773,5.398555,5.634624,0.000000,5.649428,5.645735,5.627151
14666,0.959294,4.267022,3.194454,2.333350,1.947537,3.224635,0.646372,0.948994,1.580148,1.289031,...,4.937526,1.744021,0.696448,0.810386,4.335077,1.700167,5.649428,0.000000,1.119701,2.041839


In [11]:
clusters['cluster'].value_counts()

cluster
-1    9401
 0    4833
 2      87
 3      22
 1      21
 7      13
 5      10
 6      10
 4       9
 8       7
Name: count, dtype: int64

In [12]:
monthly_agg = monthly_agg.merge(clusters, 'left', 'Product_ID')
monthly_agg['cluster'] = monthly_agg['cluster'].fillna(-2)

In [13]:
monthly_agg.dropna(subset=['target'], inplace=True)

In [14]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales,Month_date,F,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_regular_rolling_sum_sales_2,...,prev_sum_sales_12,delta_sum_sales_12,prev_sum_sales_13,delta_sum_sales_13,prev_sum_sales_14,delta_sum_sales_14,prev_sum_sales_15,delta_sum_sales_15,target,cluster
0,0,0,2019-01,0.0,2019-01-01,0.0000,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
1,0,0,2019-02,0.0,2019-02-01,0.0000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2,0,0,2019-03,0.0,2019-03-01,0.0000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
3,0,0,2019-04,0.0,2019-04-01,0.0000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
4,0,0,2019-05,0.0,2019-05-01,0.0000,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459114,3,14668,2023-07,0.0,2023-07-01,1.9666,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3459115,3,14668,2023-08,0.0,2023-08-01,1.9666,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3459116,3,14668,2023-09,0.0,2023-09-01,1.9666,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
3459117,3,14668,2023-10,0.0,2023-10-01,1.9666,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1


In [15]:
# Сортируем по Month_date (по возрастанию времени)
monthly_agg.sort_values(by=['Month_date'], inplace=True)

# Пример: берём валидацию на последние 3 месяца, которые у нас есть в данных
all_months = sorted(monthly_agg['Month_date'].unique())
N_valid_months = 3
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = monthly_agg[monthly_agg['Month_date'] < valid_threshold].sample(frac=1, random_state=42)
valid_data = monthly_agg[monthly_agg['Month_date'] >= valid_threshold]

In [16]:
feature_cols = subset + ['cluster']
cat_cols = ['Company_ID', 'Product_ID']

X_train = train_data[train_data['target'] != 0][feature_cols + cat_cols]
y_train = np.log(train_data[train_data['target'] != 0]['target'])

X_valid = valid_data[valid_data['target'] != 0][feature_cols + cat_cols]
y_valid = np.log(valid_data[valid_data['target'] != 0]['target'])

# Укажем, какие столбцы считать категориальными для CatBoost
# (CatBoost умеет работать с ними напрямую, выучивая таргет-кодирование)
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]  # индексы категориальных фич

In [17]:
from category_encoders import OneHotEncoder
ohe = OneHotEncoder(cols=['cluster'])
ohe.fit(monthly_agg[feature_cols + cat_cols])

OneHotEncoder(cols=['cluster'])

In [18]:
X_train = ohe.transform(X_train)
X_valid = ohe.transform(X_valid)

In [19]:
X_train

,F,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_regular_rolling_sum_sales_2,delta_sum_sales_1,prev_sum_sales_2,rolling_sum_sales_3,delta_regular_rolling_sum_sales_3,delta_rolling_sum_sales_2,...,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,Company_ID,Product_ID
29490,2.161816,0.0,2.0,1.0,-1.0,-2.0,0.0,0.666667,-0.666667,0.333333,...,0,0,0,0,0,0,0,0,0,499
1229635,0.982905,1.0,1.0,1.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,...,0,0,0,0,0,0,0,0,1,6182
1560952,2.177115,3.0,3.0,3.0,0.0,0.0,2.0,2.666667,0.333333,0.333333,...,0,0,0,0,0,0,0,0,1,11802
1724800,0.830900,1.0,1.0,1.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,...,0,0,0,0,0,0,0,0,1,14588
3174609,2.139948,1.0,0.0,0.5,0.5,1.0,0.0,0.333333,0.666667,0.166667,...,0,0,0,0,0,0,0,0,3,9840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970545,1.248276,3.0,2.0,2.5,0.5,1.0,1.0,2.000000,1.000000,0.500000,...,0,0,0,0,0,0,0,0,3,6375
2099764,6.862059,7.0,6.0,6.5,0.5,1.0,25.0,12.666667,-5.666667,-6.166667,...,0,0,0,0,0,0,0,0,2,6274
1385092,65.680626,1.0,0.0,0.5,0.5,1.0,0.0,0.333333,0.666667,0.166667,...,0,0,0,0,0,0,0,0,1,8828
2595552,1.893790,7.0,3.0,5.0,2.0,4.0,0.0,3.333333,3.666667,1.666667,...,0,0,0,0,0,0,0,0,3,21


In [ ]:
model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.01,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=37,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=200
)

0:	learn: 1.3061092	test: 1.3243724	best: 1.3243724 (0)	total: 326ms	remaining: 54m 22s
100:	learn: 0.7415000	test: 0.7648966	best: 0.7648966 (100)	total: 15.2s	remaining: 24m 46s
200:	learn: 0.5962777	test: 0.6241966	best: 0.6241966 (200)	total: 29.8s	remaining: 24m 10s
300:	learn: 0.5557677	test: 0.5844115	best: 0.5844115 (300)	total: 44.2s	remaining: 23m 45s
400:	learn: 0.5436452	test: 0.5719993	best: 0.5719993 (400)	total: 58.6s	remaining: 23m 23s
500:	learn: 0.5393304	test: 0.5674089	best: 0.5674089 (500)	total: 1m 12s	remaining: 23m 3s
600:	learn: 0.5373828	test: 0.5654219	best: 0.5654219 (600)	total: 1m 27s	remaining: 22m 40s
700:	learn: 0.5362344	test: 0.5642427	best: 0.5642427 (700)	total: 1m 41s	remaining: 22m 25s
800:	learn: 0.5352332	test: 0.5634112	best: 0.5634112 (800)	total: 1m 55s	remaining: 22m 9s
900:	learn: 0.5344033	test: 0.5627854	best: 0.5627854 (900)	total: 2m 10s	remaining: 21m 54s
1000:	learn: 0.5336897	test: 0.5622571	best: 0.5622571 (1000)	total: 2m 24s	remai

In [ ]:
from sklearn.metrics import mean_absolute_error
scale_factor = (monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12']
                .groupby(['Company_ID', 'Product_ID'])['sum_sales']
                .agg(lambda x: int(x.sum() > 0)).sum() * 3) / len(sample_submission)

y_pred_valid = np.exp(model.predict(X_valid))
mae_valid = mean_absolute_error(np.exp(y_valid), y_pred_valid)
print("Raw MAE (valid):    ", mae_valid)
print("Scaled MAE (valid): ", mae_valid * scale_factor)

In [19]:
def predict_next_month(df, model):
    df = df.copy()
    df['Month_date'] = pd.to_datetime(df['Month'], format='%Y-%m')
    df['Month_pred'] = (df['Month_date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').astype(str)
    df = df.sort_values(by='Month_date')

    gb = df.groupby(['Company_ID', 'Product_ID'])
    for i in range(n_lags + 1):
        df[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
        if i > 0:
            if i + 1 <= 9:
                feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
                df[f'rolling_sum_sales_{i + 1}'] = df[feats_to_mean].mean(skipna=False, axis=1)
                df[f'delta_regular_rolling_sum_sales_{i + 1}'] = df['prev_sum_sales_0'] - df[f'rolling_sum_sales_{i + 1}']
                if i > 1:
                    df[f'delta_rolling_sum_sales_{i}'] = df[f'rolling_sum_sales_{i}'] - df[f'rolling_sum_sales_{i + 1}']
            df[f'delta_sum_sales_{i}'] = df['prev_sum_sales_0'] - df[f'prev_sum_sales_{i}']

    df['Target'] = gb['sum_sales'].shift(-1)

    df = df.merge(clusters, 'left', 'Product_ID')
    df['cluster'] = df['cluster'].fillna(-2)

    to_predict = df['Target'].isna()
    df.loc[to_predict, 'Target'] = np.exp(model.predict(df.loc[to_predict, feature_cols + cat_cols])).round()
    df.loc[to_predict, 'Target'] = df.loc[to_predict, 'Target'].apply(lambda x: max(x, 0))
    return df.loc[to_predict, ['Company_ID', 'Product_ID', 'Month_pred', 'Target']]

In [ ]:
mag = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') < valid_threshold].copy()
preds = []
for i in range(N_valid_months):
    p = predict_next_month(mag, model)
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    preds.append(p)
    mag = pd.concat([mag, p], axis=0)

preds = pd.concat(preds, axis=0)
valid_data = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') >= valid_threshold].copy()
merged = valid_data.merge(preds, on=['Company_ID', 'Product_ID', 'Month'], suffixes=('_actual', '_pred'))
raw_mae = mean_absolute_error(merged['sum_sales_actual'], merged['sum_sales_pred'])
print("Рекурсивный прогноз - raw MAE (valid):    ", raw_mae)
print("Рекурсивный прогноз - scaled MAE (valid): ", raw_mae * scale_factor)

In [ ]:
full_train = monthly_agg.dropna(subset=['target']).sample(frac=1, random_state=42)
X_full = full_train[full_train['target'] != 0][feature_cols + cat_cols]
y_full = np.log(full_train[full_train['target'] != 0]['target'])

final_model = CatBoostRegressor(
    iterations=3961,
    learning_rate=0.01,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=42,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

In [ ]:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[1]).astype(int)
sample_submission['Month_pred'] = sample_submission['Id'].apply(lambda x: x.split('_')[2]).astype(str)
sample_submission

In [24]:
preds = []
mag = monthly_agg_saved.copy()
for i in range(3):
    p = predict_next_month(mag, final_model)
    preds.append(p.copy())
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    mag = pd.concat([mag, p], axis=0)
preds = pd.concat(preds, axis=0)
sample_submission = sample_submission.drop(columns=['Target'], axis=1).merge(preds, 'left', on=['Company_ID', 'Product_ID', 'Month_pred'])

In [ ]:
to_mult = monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12'].groupby(['Company_ID', 'Product_ID'])['sum_sales'].agg(lambda x: int(x.sum() > 0)).reset_index()
for i in tqdm.tqdm(to_mult.index):
    sample_submission.loc[(sample_submission['Company_ID'] == to_mult.loc[i, 'Company_ID']) &
                          (sample_submission['Product_ID'] == to_mult.loc[i, 'Product_ID']), 'Target'] *= to_mult.loc[i, 'sum_sales']

In [26]:
# sample_submission['Target'] = np.round(0.9 * sample_submission['Target'])

In [ ]:
sample_submission[['Id', 'Target']]

In [28]:
sample_submission[['Id', 'Target']].to_csv('data/submissions/pivo.csv', index=False)